# Step using amortized VBD and visualize

In [1]:
import os
import torch
import mediapy

from pathlib import Path
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir.parent)

from gpudrive.env.config import EnvConfig, RenderConfig
from gpudrive.env.dataset import SceneDataLoader
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.datatypes.trajectory import VBDTrajectory
from gpudrive.visualize.utils import img_from_fig
import madrona_gpudrive

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [ ]:
#Init GPUDrive env
device = "cpu"

env_config = EnvConfig(
    dynamics_model="state", # Use state-based dynamics model
    dist_to_goal_threshold=1e-5, # Trick to make sure the agents don't disappear when they reach the goal
    init_mode = 'wosac_eval',
    max_controlled_agents=32,
    goal_behavior='ignore',
    guidance_mode='vbd_amortized'
)
        
# Make env
gpudrive_env = GPUDriveTorchEnv(
    config=env_config,
    data_loader = SceneDataLoader(
        root="data/processed/wosac/validation/json",
        batch_size=1,
        dataset_size=1,
        file_prefix=""
    ),
    render_config=RenderConfig(),
    max_cont_agents=32, # Maximum number of agents to control per scene
    device=device,
)

Smoothing guidance data: 100%|██████████| 1/1 [00:00<00:00, 61.80it/s]


In [3]:
# Action tensor to step through simulation
predicted_actions = torch.zeros((gpudrive_env.num_worlds, gpudrive_env.max_agent_count, madrona_gpudrive.kTrajectoryLength, 10))

vbd_trajectories = VBDTrajectory.from_tensor(
    gpudrive_env.sim.vbd_trajectory_tensor(),
    device=device,
)

predicted_actions[:, :, :, 0] = vbd_trajectories.pos_x.squeeze(-1)
predicted_actions[:, :, :, 1] = vbd_trajectories.pos_y.squeeze(-1)
predicted_actions[:, :, :, 3] = vbd_trajectories.yaw.squeeze(-1)
predicted_actions[:, :, :, 4] = vbd_trajectories.vel_x.squeeze(-1)
predicted_actions[:, :, :, 5] = vbd_trajectories.vel_y.squeeze(-1)

sim_state_frames = []
fig = gpudrive_env.vis.plot_simulator_state(
    time_steps=[0],
    env_indices=[0],
    zoom_radius=120,
    plot_guidance_pos_xy=True,
)[0]
# Save initial state
sim_state_frames.append(img_from_fig(fig))

# Get expert actions
# expert_actions, _, _, _ = gpudrive_env.get_expert_actions()
# Step through simulation
for t in range(env_config.episode_len):
    gpudrive_env.step_dynamics(predicted_actions[:, :, t, :])
    fig = gpudrive_env.vis.plot_simulator_state(
        time_steps=[t],
        env_indices=[0],
        zoom_radius=120,
        plot_guidance_pos_xy=True,
    )[0]
    sim_state_frames.append(img_from_fig(fig))

In [4]:
controlled_agent_mask = gpudrive_env.cont_agent_mask[0, :]
controlled_agent_mask, controlled_agent_mask.sum().item()

(tensor([ True,  True,  True, False,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False]),
 14)

In [5]:
trajectories = vbd_trajectories.pos_xy[0, controlled_agent_mask]
trajectories

tensor([[[ -5.6777,  33.1743],
         [ -6.0010,  33.1763],
         [ -6.3291,  33.1802],
         ...,
         [-24.5342,  33.6284],
         [-24.6396,  33.6313],
         [-24.7119,  33.6333]],

        [[ -0.9980,  21.9150],
         [ -1.0020,  22.0342],
         [ -0.9746,  22.1816],
         ...,
         [  1.0430,  32.8506],
         [  1.0645,  32.9722],
         [  1.0879,  33.0996]],

        [[-47.2080,  40.1299],
         [-47.1768,  40.0386],
         [-47.1357,  39.9395],
         ...,
         [-40.8311,  31.8291],
         [-40.7695,  31.7153],
         [-40.7090,  31.6021]],

        ...,

        [[  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         ...,
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000]],

        [[-78.5566,  33.8350],
         [-78.4814,  33.9312],
         [-78.4121,  34.0332],
         ...,
         [-76.4219,  41.7549],
         [-76.3252,  41.8037],
         

In [ ]:
mediapy.write_video('gpudrive/integrations/vbd/viz/eval.gif', sim_state_frames, fps=20, codec="gif")